In [ ]:
"""Project Summary

Step 1: Scope the Project and Gather Data
Step 2: Explore and Assess the Data
Step 3: Define the Data Model
Step 4: Run ETL to Model the Data
Step 5: Complete Project Write Up

"""

In [ ]:
'''
# Step 1: Scope the Project and Gather Data

-- From Toronto datasets collected, we will extract various data points on apartment buildings in the Toronto area
such income, type of building, and ratings. In the end, We will build a dashboard to show those found data points
to understand the potential area we will place our distribution center. 
'''

In [56]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
import psycopg2
import uuid
import re
import os
import configparser
import boto3

# Configuration

In [55]:
# Get the Spark session
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

spark

In [62]:
# 1.1 reading in data for development applications
dev_application_df = (spark.read
  .option("header", True)
  .csv("Development_Applications_Data.csv")
)
dev_application_df.limit(3).toPandas()


,_id,APPLICATION#,APPLICATION_TYPE,DATE_SUBMITTED,DESCRIPTION,HEARING_DATE,POSTAL,REFERENCE_FILE#,STATUS,STREET_DIRECTION,STREET_NAME,STREET_NUM,STREET_TYPE,X,Y
0,540631,17 260753 STE 28 SA,SA,2017-11-10,Site Plan Control application for a proposed 1...,None,,None,Closed,E,KING,65,ST,314829.237,4834177.49
1,540632,18 189938 STE 22 OZ,OZ,2018-07-06,The City has received applications to amend th...,None,M4T,None,Under Review,,YONGE,1506,ST,313272.284,4838553.3
2,540633,18 189996 NNY 25 MV,MV,2018-07-06,To construct a new residential dwelling with a...,2018-08-30,M4N,A0513/18NY,Approved,,TEDDINGTON PARK,50,AVE,312715.229,4843548.456


In [63]:
# 1.2 reading in data for ward demographics
ward_demo_df = (spark.read
    .option("header", True)
    .csv("pdf_census.csv")
)
dev_application.limit(3).toPandas()

,Ward Population,Ward
0,Median Age 40.6,Ward 20
1,Median Age 40P,Ward 15
2,Median Age 35.6,Ward 11


In [72]:
# 1.3 reading in data for apartment building evaluation
build_eval_df = (spark.read
  .option("header", True)
  .json("Apartment_Building_Evaluation.json")
)
build_eval_df.limit(3).toPandas()


,BALCONY_GUARDS,CONFIRMED_STOREYS,CONFIRMED_UNITS,ELEVATORS,ENTRANCE_DOORS_WINDOWS,ENTRANCE_LOBBY,EVALUATION_COMPLETED_ON,EXTERIOR_CLADDING,EXTERIOR_GROUNDS,EXTERIOR_WALKWAYS,...,SCORE,SECURITY,SITE_ADDRESS,STAIRWELLS,STORAGE_AREAS_LOCKERS,WARD,WATER_PEN_EXT_BLDG_ELEMENTS,YEAR_BUILT,YEAR_REGISTERED,_id
0,NaN,7.0,34.0,4.0,4.0,4.0,03/12/2019,3.0,4.0,4.0,...,78.0,5.0,49 ST CLAIR AVE W,4.0,NaN,12,4.0,1930.0,2017.0,6906
1,NaN,4.0,24.0,NaN,3.0,3.0,03/05/2019,3.0,3.0,3.0,...,73.0,5.0,320 AVENUE RD,4.0,NaN,12,4.0,1925.0,2017.0,6907
2,4.0,7.0,68.0,4.0,4.0,4.0,03/08/2019,3.0,4.0,4.0,...,77.0,5.0,7 EDMUND AVE,3.0,4.0,12,4.0,1928.0,2017.0,6908


In [65]:
'''Step 2: Explore and Assess the Data

Explore the Data
Identify data quality issues, like missing values, duplicate data, etc.

Cleaning Steps

# 2.1 development applications
- removing unneseccary columns
- creating a complete address
- matching address with Ward

# 2.2 ward demographics
- remove "Median Age" and "Ward" from rows
- round median age

# 2.3 apartment building evaluation
- rows with no BALCONY_GUARDS should be removed
- Evavator column, NAN should be replaced with 0 and 1 if there is an elevator

'''


'Step 2: Explore and Assess the Data\n\nExplore the Data\nIdentify data quality issues, like missing values, duplicate data, etc.\n\nCleaning Steps\n\n# 2.1 development applications\n- removing unneseccary columns\n- creating a complete address\n- matching address with Ward\n\n# 2.2 ward demographics\n- remove "Median Age" and "Ward" from rows\n- round median age\n\n# 2.3 apartment building evaluation\n- rows with no BALCONY_GUARDS should be removed\n- Evavator column, NAN should be replaced with 0 and 1 if there is an elevator\n\n'

In [ ]:
'''2.1 development applications
- removing unneseccary columns
- creating a complete address
- matching address with Ward
'''


In [ ]:
'''2.2 ward demographics
- remove "Median Age" and "Ward" from rows
- round median age
'''

In [ ]:
''' 2.3 apartment building evaluation
- rows with no BALCONY_GUARDS should be removed
- Evavator column, NAN should be replaced with 0 and 1 if there is an elevator
'''

In [ ]:
''' Step 3: Define the Data Model
'''

In [70]:
# Need to create a temp veiw to run spark sql
dev_application_df.createOrReplaceTempView("dev_application")
ward_demo_df.createOrReplaceTempView("ward_demo")
build_eval_df.createOrReplaceTempView("build_eval")

In [71]:
'''Step 4: Run Pipelines to Model the Data

4.1 Create the data model

# 4.1 development applications
'''
[_id , SUBMITTED, DESCRIPTION, STATUS, STREET_DIRECTION, STREET_NAME, STREET_NUM, STREET_TYPE]

dev_application_table = spark.sql("""
select
      _id,
      DATE_SUBMITTED,
      DESCRIPTION,
      STATUS,
      STREET_DIRECTION,
      STREET_NAME,
      STREET_NUM,
      STREET_TYPE,
from dev_application
""")
dev_application_table.limit(2).toPandas()

NameError: name '_id' is not defined

In [ ]:
# Writing tables in s3
table_name = "dev_application_table"
parquet_file_name = temp_bucket + table_name + ".parquet"
dev_application_table.write.parquet("s3a://" + parquet_file_name)

In [ ]:
# 4.2 ward demographics


In [ ]:
# 4.3 apartment building evaluation


In [ ]:
'''4.4 Data dictionary

Create a data dictionary for your data model. 
For each field, provide a brief description of what the data is and where it came from. 

'''

In [ ]:
# Clean up 


In [ ]:
'''
Step 5: Complete Project Write Up


'''